In [65]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [66]:
!pip install missingpy

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math

In [87]:
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [88]:
import sys
import numpy as np
import pandas as pd

from missingpy import MissForest
from dsan import Imputer as DSANImputer
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [89]:
seed = 128
set_seed(seed)

In [90]:
def get_imputer(config):
    
    model = config['model']
    
    if model == 'DSAN':
        imputer = DSANImputer(rep_dim=config['rep_dim'],
                num_heads=config['num_heads'],
                n_hidden=config['n_hidden'],
                lr=config['lr'],
                weight_decay=config['weight_decay'],
                batch_size=config['batch_size'],
                epochs=config['epochs'],
                noise_percent=config['noise_percent'],
                stopped_epoch=config['stopped_epoch'])
    
    elif model == 'MissForest':
        imputer = MissForest(random_state=0)
        
    elif model == 'Statistics':
        imputer = None
        
    return imputer

In [91]:
'''def get_data(data_name):
    if data_name in ['titanic', 'breast', 'adult', 'bank']:
        # Existing code to handle these datasets
        pass
    elif data_name == 'online':
        # Load your new dataset without a target column
        data = pd.read_csv('/Users/emondemoniac/Desktop/TabNet_PyTorch/Heart_disease/online.csv')  # Adjust the path to your dataset

        # Preprocess your dataset here (handle missing values, encode categories, etc.)
        X_origin = data  # Use the entire dataset as there's no target column to drop

        # Identify numerical and categorical variables
        num_vars = list(X_origin.select_dtypes(include=['int64', 'float64']).columns)
        cat_vars = list(X_origin.select_dtypes(include=['object', 'category']).columns)
        n_col = X_origin.shape[1]

        return X_origin, n_col, num_vars, cat_vars
    else:
        raise ValueError('Unsupported dataset. Please choose from supported datasets or add your new dataset in get_data function.')
'''

"def get_data(data_name):\n    if data_name in ['titanic', 'breast', 'adult', 'bank']:\n        # Existing code to handle these datasets\n        pass\n    elif data_name == 'online':\n        # Load your new dataset without a target column\n        data = pd.read_csv('/Users/emondemoniac/Desktop/TabNet_PyTorch/Heart_disease/online.csv')  # Adjust the path to your dataset\n\n        # Preprocess your dataset here (handle missing values, encode categories, etc.)\n        X_origin = data  # Use the entire dataset as there's no target column to drop\n\n        # Identify numerical and categorical variables\n        num_vars = list(X_origin.select_dtypes(include=['int64', 'float64']).columns)\n        cat_vars = list(X_origin.select_dtypes(include=['object', 'category']).columns)\n        n_col = X_origin.shape[1]\n\n        return X_origin, n_col, num_vars, cat_vars\n    else:\n        raise ValueError('Unsupported dataset. Please choose from supported datasets or add your new dataset in 

In [92]:
def experiment_run(data_name, config, test=False):
    # Retrieve dataset
    dataset = get_data(data_name)
    print(f"Retrieved dataset for {data_name}")

    # Limit to 1000 samples if test flag is true
    if test:
        dataset = (dataset[0][:1000], dataset[1], dataset[2], dataset[3])
        print("Limited dataset to 1000 samples for testing")

    k = 5
    percent_missing_lst = [p for p in range(5, 25, 5)]
    X_origin, n_col, num_vars, cat_vars = dataset

   # Apply category mapping
    X_origin = category_mapping(X_origin, cat_vars)
    print("Applied category mapping")

    # Check if X_origin is modified in-place or replaced
    if X_origin is not None:
        print("X_origin is replaced with a new version where categorical variables have been mapped to numerical values.")
    else:
        print("X_origin is modified in-place, with categorical variables mapped to numerical values.")

    # Print data size
    print("Data Size: {}".format(X_origin.shape))


    n_sample = X_origin.shape[0]
    idx_lst = list(range(n_sample))
    np.random.shuffle(idx_lst)

    # k-fold validation setup
    n_valid = len(idx_lst) // k
    print(f"Starting k-fold validation with k={k}")

    results = {}
    for percent_missing in percent_missing_lst:
        result = []
        print(f"\nProcessing {percent_missing}% missing data")

        for it, n in enumerate(range(0, n_sample, n_valid)):
            print(f"\nFold {it + 1}")
            valid_idx = idx_lst[n: n+n_valid]

            if len(valid_idx) < n_valid:
                print("Insufficient data for validation, breaking loop")
                break

            train_idx = list(set(idx_lst) - set(valid_idx))
            X = X_origin[train_idx, :]
            X_test = X_origin[valid_idx, :]

            # Generating incomplete data
            missing_mask = generate_missing_mask(X, percent_missing=percent_missing)
            X_incomplete = X.copy()
            X_incomplete[missing_mask] = np.nan
            print("Generated incomplete data with missing values for fold {}:".format(it+1))
            print("Original data with missing values (subset):")
            print(X_incomplete[:5])  # Adjust the number of rows as needed

            # Imputation
            imputer = get_imputer(config)
            X_imputed = imputer.fit_transform(X_incomplete, cat_vars=cat_vars)
            print("Completed imputation for fold {}:".format(it+1))

            X_res = X.copy()
            X_res[missing_mask] = X_imputed[missing_mask]
            print("X_res with imputed values (subset):")
            print(X_res[:5])  # Adjust the number of rows as needed

            # Calculate metrics
            metric = cal_metric_numpy(X_res, X, missing_mask, num_vars, cat_vars)
            score = ex_classify(data_name=data_name, train_array=X_res, test_array=X_test, num_vars=num_vars, cat_vars=cat_vars)
            
            metric['clf_aucroc'] = round(score, 4)
            result.append(metric)
            print("Metrics for this fold:", metric)

        # Average results
        avg_result = dict()
        for key in result[0].keys():
            value = np.mean([m[key] for m in result])
            avg_result[key] = value 
            print(f"Average {key} for {percent_missing}% missing data: {value}")

        results[percent_missing] = avg_result

    print("Completed all iterations")
    return results


In [93]:
config = {
    'model': 'DSAN',
    'rep_dim': 32,
    'num_heads': 8,
    'n_hidden': 2,
    'lr': 3e-3,
    'weight_decay': 1e-5,
    'batch_size': 128,
    'epochs': 15,
    'noise_percent': 10,
    'stopped_epoch': 10
}

In [94]:
data_name_lst = ['breast']

result_dir = './results'
os.makedirs(result_dir)

for data_name in data_name_lst:
    results = experiment_run(data_name, config, test=True)
    results['config'] = config
    results['dataset'] = data_name
    result_path = os.path.join(result_dir, '{}.json'.format(data_name))
    write_json(results, result_path)

Retrieved dataset for breast
Limited dataset to 1000 samples for testing
Applied category mapping
X_origin is replaced with a new version where categorical variables have been mapped to numerical values.
Data Size: (569, 31)
Starting k-fold validation with k=5

Processing 5% missing data

Fold 1
Generated incomplete data with missing values for fold 1:
Original data with missing values (subset):
[[1.969e+01 2.125e+01 1.300e+02 1.203e+03 1.096e-01 1.599e-01 1.974e-01
  1.279e-01 2.069e-01 5.999e-02 7.456e-01 7.869e-01 4.585e+00 9.403e+01
        nan 4.006e-02 3.832e-02       nan 2.250e-02 4.571e-03 2.357e+01
  2.553e+01 1.525e+02 1.709e+03 1.444e-01 4.245e-01 4.504e-01 2.430e-01
  3.613e-01       nan 0.000e+00]
 [1.142e+01 2.038e+01 7.758e+01 3.861e+02 1.425e-01 2.839e-01 2.414e-01
  1.052e-01 2.597e-01 9.744e-02 4.956e-01 1.156e+00 3.445e+00 2.723e+01
  9.110e-03 7.458e-02 5.661e-02 1.867e-02       nan 9.208e-03 1.491e+01
  2.650e+01 9.887e+01 5.677e+02 2.098e-01 8.663e-01 6.869e-01 2.

100%|███████████████████████████████████████████| 15/15 [00:09<00:00,  1.59it/s]


Completed imputation for fold 1:
X_res with imputed values (subset):
[[1.96900000e+01 2.12500000e+01 1.30000000e+02 1.20300000e+03
  1.09600000e-01 1.59900000e-01 1.97400000e-01 1.27900000e-01
  2.06900000e-01 5.99900000e-02 7.45600000e-01 7.86900000e-01
  4.58500000e+00 9.40300000e+01 7.53093045e-03 4.00600000e-02
  3.83200000e-02 1.77643672e-02 2.25000000e-02 4.57100000e-03
  2.35700000e+01 2.55300000e+01 1.52500000e+02 1.70900000e+03
  1.44400000e-01 4.24500000e-01 4.50400000e-01 2.43000000e-01
  3.61300000e-01 8.04896951e-02 0.00000000e+00]
 [1.14200000e+01 2.03800000e+01 7.75800000e+01 3.86100000e+02
  1.42500000e-01 2.83900000e-01 2.41400000e-01 1.05200000e-01
  2.59700000e-01 9.74400000e-02 4.95600000e-01 1.15600000e+00
  3.44500000e+00 2.72300000e+01 9.11000000e-03 7.45800000e-02
  5.66100000e-02 1.86700000e-02 2.09250785e-02 9.20800000e-03
  1.49100000e+01 2.65000000e+01 9.88700000e+01 5.67700000e+02
  2.09800000e-01 8.66300000e-01 6.86900000e-01 2.57500000e-01
  6.63800000e-0

100%|███████████████████████████████████████████| 15/15 [00:09<00:00,  1.55it/s]


Completed imputation for fold 2:
X_res with imputed values (subset):
[[1.79900000e+01 1.03800000e+01 1.22800000e+02 1.00100000e+03
  1.18400000e-01 2.77600000e-01 3.00100000e-01 1.47100000e-01
  2.41900000e-01 7.87100000e-02 1.09500000e+00 9.05300000e-01
  8.58900000e+00 1.53400000e+02 6.39900000e-03 4.90400000e-02
  5.37300000e-02 1.58700000e-02 3.00300000e-02 6.19300000e-03
  2.53800000e+01 1.73300000e+01 1.84600000e+02 2.01900000e+03
  1.62200000e-01 6.65600000e-01 7.11900000e-01 2.65400000e-01
  4.60100000e-01 1.18900000e-01 0.00000000e+00]
 [2.05700000e+01 1.77700000e+01 1.32900000e+02 1.32600000e+03
  8.47400000e-02 7.86400000e-02 8.69000000e-02 7.01700000e-02
  1.81200000e-01 5.66700000e-02 5.43500000e-01 7.33900000e-01
  3.39800000e+00 7.40800000e+01 5.22500000e-03 1.30800000e-02
  1.86000000e-02 1.34000000e-02 1.38900000e-02 3.53200000e-03
  2.49900000e+01 2.34100000e+01 1.58800000e+02 1.95600000e+03
  1.23800000e-01 1.86600000e-01 2.41600000e-01 1.86000000e-01
  3.04255724e-0

...

Completed imputation for fold 3:
X_res with imputed values (subset):
[[1.79900000e+01 1.03800000e+01 1.22800000e+02 1.00100000e+03
  1.18400000e-01 2.77600000e-01 3.00100000e-01 1.47100000e-01
  2.41900000e-01 7.87100000e-02 1.09500000e+00 9.05300000e-01
  8.58900000e+00 1.53400000e+02 7.50361383e-03 4.90400000e-02
  5.37300000e-02 2.42006406e-02 3.00300000e-02 6.19300000e-03
  2.53800000e+01 1.73300000e+01 1.84600000e+02 2.01900000e+03
  1.62200000e-01 6.65600000e-01 7.11900000e-01 2.65400000e-01
  4.60100000e-01 1.18900000e-01 0.00000000e+00]
 [2.05700000e+01 1.77700000e+01 1.32900000e+02 1.32600000e+03
  8.47400000e-02 7.86400000e-02 2.09777221e-01 1.03024796e-01
  1.81200000e-01 5.95779531e-02 5.43500000e-01 7.33900000e-01
  3.39800000e+00 7.40800000e+01 5.22500000e-03 1.30800000e-02
  1.86000000e-02 1.34000000e-02 1.38900000e-02 3.53200000e-03
  2.49900000e+01 2.34100000e+01 1.56843170e+02 1.95600000e+03
  1.20340578e-01 1.86600000e-01 5.90339899e-01 1.86000000e-01
  2.75000000e-0

100%|███████████████████████████████████████████| 15/15 [00:10<00:00,  1.44it/s]


Completed imputation for fold 4:
X_res with imputed values (subset):
[[1.79900000e+01 1.03800000e+01 1.44487289e+02 1.27189429e+03
  1.18400000e-01 2.77600000e-01 3.00100000e-01 1.47100000e-01
  1.81747660e-01 6.03736527e-02 1.09500000e+00 9.05300000e-01
  8.58900000e+00 1.53400000e+02 6.39900000e-03 4.90400000e-02
  5.37300000e-02 1.58700000e-02 3.00300000e-02 4.61815251e-03
  2.53800000e+01 1.73300000e+01 1.84600000e+02 2.01900000e+03
  1.62200000e-01 6.65600000e-01 5.41557610e-01 2.65400000e-01
  4.60100000e-01 1.18900000e-01 0.00000000e+00]
 [2.05700000e+01 1.77700000e+01 1.32900000e+02 1.25030017e+03
  8.47400000e-02 7.86400000e-02 8.69000000e-02 7.01700000e-02
  1.81200000e-01 5.66700000e-02 5.43500000e-01 7.33900000e-01
  3.39800000e+00 7.40800000e+01 5.22500000e-03 1.30800000e-02
  3.01122684e-02 1.34000000e-02 1.96176134e-02 3.53200000e-03
  2.49900000e+01 2.34100000e+01 1.58800000e+02 1.95600000e+03
  1.23800000e-01 1.86600000e-01 2.41600000e-01 1.86000000e-01
  2.75000000e-0

100%|███████████████████████████████████████████| 15/15 [00:13<00:00,  1.11it/s]


Completed imputation for fold 5:
X_res with imputed values (subset):
[[1.79900000e+01 1.03800000e+01 1.22800000e+02 1.00100000e+03
  1.18400000e-01 2.77600000e-01 3.00100000e-01 1.47100000e-01
  2.41900000e-01 7.87100000e-02 1.09500000e+00 9.05300000e-01
  8.58900000e+00 1.53400000e+02 6.39900000e-03 4.90400000e-02
  5.37300000e-02 1.58700000e-02 3.00300000e-02 6.19300000e-03
  2.53800000e+01 1.73300000e+01 1.84600000e+02 2.01900000e+03
  1.62200000e-01 6.65600000e-01 7.11900000e-01 2.24438846e-01
  4.60100000e-01 1.18900000e-01 0.00000000e+00]
 [2.05700000e+01 1.77700000e+01 1.32900000e+02 1.09435522e+03
  9.43592861e-02 7.86400000e-02 8.69000000e-02 7.87259713e-02
  1.81200000e-01 5.66700000e-02 5.43500000e-01 1.03549528e+00
  3.28892612e+00 7.40800000e+01 5.93996886e-03 1.30800000e-02
  1.86000000e-02 1.34000000e-02 1.38900000e-02 3.53200000e-03
  2.49900000e+01 2.68568954e+01 1.58800000e+02 1.44744263e+03
  1.36731237e-01 1.86600000e-01 2.41600000e-01 1.86000000e-01
  2.88275927e-0